In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
%pip --disable-pip-version-check install flair
%pip --disable-pip-version-check install flask-ngrok

     |████████████████████████████████| 286kB 6.1MB/s 
     |████████████████████████████████| 19.7MB 1.3MB/s 
     |████████████████████████████████| 1.2MB 39.4MB/s 
     |████████████████████████████████| 776.8MB 23kB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 798kB 39.4MB/s 
     |████████████████████████████████| 2.1MB 29.8MB/s 
     |████████████████████████████████| 983kB 33.6MB/s 
     |████████████████████████████████| 71kB 7.4MB/s 
     |████████████████████████████████| 901kB 28.4MB/s 
     |████████████████████████████████| 3.3MB 42.3MB/s 
  Created wheel for gdown: filename=gdown-3.12.2-cp37-none-any.whl size=9693 sha256=79a9423102b1a382c667485e89211c65adb6f464b0c8c0d002b28f0145bf05dd
  Stored in directory: /root/.cache/pip/wheels/81/d0/d7/d9983facc6f2775411803e0e2d30ebf98efbf2fc6e57701e09
Successfully built gdown
  Created wheel for segtok: filename

In [3]:
!python -c "import flask; print(flask.__version__)"

1.1.2


In [4]:
# Importing the libraries
import os
import pickle
import flask
import json
import random
import time
from tqdm import tqdm
from scipy.spatial.distance import cosine
from flair.data import Sentence
from flair.embeddings import WordEmbeddings, DocumentPoolEmbeddings

from flask import Flask, flash, request, redirect, url_for
from werkzeug.utils import secure_filename
from flask import Flask, render_template, request
app = flask.Flask(__name__)

# Setting the current working directory
os.chdir('/content/drive/MyDrive/Projects/AI-ML/Capstone/')
UPLOAD_FOLDER = '/content/drive/MyDrive/Projects/AI-ML/Capstone/'

embeddings = DocumentPoolEmbeddings([WordEmbeddings('en')],pooling='mean',)

!mkdir templates -p

2021-04-26 02:39:36,902 https://flair.informatik.hu-berlin.de/resources/embeddings/token/en-fasttext-news-300d-1M.vectors.npy not found in cache, downloading to /tmp/tmpwqv4bro4


100%|██████████| 1200000128/1200000128 [00:47<00:00, 25290094.37B/s]

2021-04-26 02:40:24,769 copying /tmp/tmpwqv4bro4 to cache at /root/.flair/embeddings/en-fasttext-news-300d-1M.vectors.npy


2021-04-26 02:40:29,189 removing temp file /tmp/tmpwqv4bro4
2021-04-26 02:40:29,842 https://flair.informatik.hu-berlin.de/resources/embeddings/token/en-fasttext-news-300d-1M not found in cache, downloading to /tmp/tmpbq0srptd


100%|██████████| 54600983/54600983 [00:03<00:00, 16999693.46B/s]

2021-04-26 02:40:33,529 copying /tmp/tmpbq0srptd to cache at /root/.flair/embeddings/en-fasttext-news-300d-1M


2021-04-26 02:40:33,605 removing temp file /tmp/tmpbq0srptd


In [5]:
def prepare_embeddings(input_file,output_file):
  print(1)
  global embeddings
  embedded_intent_dict = {}
  with open(input_file) as file:
    intent_dict = json.load(file)
  for intent,examples in tqdm(intent_dict.items()):
    embedded_intent_dict[intent] = []
    for example in examples:
      sentence = Sentence(example)
      embeddings.embed(sentence)
      embedded_intent_dict[intent].append(sentence.embedding.detach().numpy())

  if not os.path.exists(os.path.dirname(output_file)):
      os.makedirs(os.path.dirname(output_file))

  pickle.dump(embedded_intent_dict,open( output_file, "wb+"))

In [6]:
def answer(message,embeddings_file,anwsers_file):
    global embeddings
    with open(embeddings_file, 'rb') as file:
        embedded_dict = pickle.load(file)
    message_sentence = Sentence(message)
    embeddings.embed(message_sentence)
    message_vector = message_sentence.embedding.detach().numpy()
    best_intent = ""
    best_score = 1
    for intent, examples in embedded_dict.items():
        for example in examples:
            score = cosine(message_vector, example)
            if(score<best_score):
                best_score = score
                best_intent = intent
    with open(anwsers_file) as file:
        anwsers_dict = json.load(file)
    if(best_intent in anwsers_dict):
        return random.choice(anwsers_dict[best_intent])
    else:
        return "I am sorry! Please rephrase your query to answer you."

In [7]:
# create Pickle file for runtime use.
input_file   = "/content/drive/MyDrive/Projects/AI-ML/Capstone/intents.json"
output_file  = "/content/drive/MyDrive/Projects/AI-ML/Capstone/intents_pkl.pickle"

prepare_embeddings(input_file, output_file)

1


100%|██████████| 37/37 [00:00<00:00, 344.50it/s]


In [21]:
def chat_response(input_message):  
  anwsers_file = "/content/drive/MyDrive/Projects/AI-ML/Capstone/answer.json"
  output_file  = "/content/drive/MyDrive/Projects/AI-ML/Capstone/intents_pkl.pickle"
  return str(answer(input_message,embeddings_file=output_file,anwsers_file=anwsers_file))

In [38]:
%%writefile templates/industrialsafety_chat.html

<!DOCTYPE html>

<html>
<head>
	<link href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/4.7.0/css/font-awesome.css" type="text/css" rel="stylesheet">
	<link href="https://maxcdn.bootstrapcdn.com/bootstrap/4.1.1/css/bootstrap.min.css" rel="stylesheet" id="bootstrap-css">
	<script src="https://maxcdn.bootstrapcdn.com/bootstrap/4.1.1/js/bootstrap.min.js"></script>
  <script src="https://ajax.googleapis.com/ajax/libs/jquery/3.3.1/jquery.min.js"></script>
</head>
<body>
<div class="container">
	<h3 class=" text-center">Messaging</h3>
	<div class="messaging">
		<div class="mesgs">
			<div id="chatbox" class="msg_history">
				<div class="incoming_msg">
					<div class="incoming_msg_img"> <img src="https://ptetutorials.com/images/user-profile.png" alt="chatbot"> </div>
					<div class="received_msg">
						<div class="received_withd_msg">
						  <p>Welcome to the Chat Service! I can help you on Industrial Safety and its effectiveness.</p>
						</div>
					</div>
				</div>
				<div class="outgoing_msg">
					<div class="sent_msg"></div>
				</div>
			</div>
			<div class="type_msg">
				<div id="userInput" class="input_msg_write">
				  <input id="textInput" type="text" class="write_msg" name="msg" placeholder="Type a message">
				  <button id="buttonInput" class="msg_send_btn" type="submit"><i class="fa fa-paper-plane-o" aria-hidden="true"></i></button>
				</div>
			</div>
		</div>
	</div>
</div>
<script>
    function getResponse() {
        let userText = $("#textInput").val();
        let userHtml = '<div class="outgoing_msg"><div class="sent_msg"><p>' + userText + '</p></div></div>';
        $("#textInput").val("");
        $("#chatbox").append(userHtml);

        $.get("/get", { msg: userText }).done(function(data) {
          let ansHtml = '<div class="incoming_msg"><div class="incoming_msg_img"><img src="https://ptetutorials.com/images/user-profile.png" alt="chatbot"> </div><div class="received_msg"><div  class="received_withd_msg"><p>' + data + '</p></div></div></div>';
          $("#chatbox").append(ansHtml);
          document.getElementById('userInput').scrollIntoView({block: 'start', behavior: 'smooth'});

          var objDiv = document.getElementById('chatbox');
          objDiv.scrollTop = objDiv.scrollHeight;
        });
    }
	$("#textInput").keypress(function(e) {
    //if enter key is pressed
        if(e.which == 13) {
            getResponse();
        }
    });
    $("#buttonInput").click(function() {
        getResponse();
    });
	
	n =  new Date();
	y = n.getFullYear();
	m = n.getMonth() + 1;
	d = n.getDate();
	//document.getElementById("date").innerHTML = m + "/" + d + "/" + y;
	
</script>
</body>
</html>

<style>
.container{max-width:1170px; margin:auto;}
img{ max-width:100%;}
.inbox_people {
  background: #f8f8f8 none repeat scroll 0 0;
  float: left;
  overflow: hidden;
  width: 40%; border-right:1px solid #c4c4c4;
}
.inbox_msg {
  border: 1px solid #c4c4c4;
  clear: both;
  overflow: hidden;
}
.top_spac{ margin: 20px 0 0;}


.recent_heading {float: left; width:40%;}
.srch_bar {
  display: inline-block;
  text-align: right;
  width: 60%;
}
.headind_srch{ padding:10px 29px 10px 20px; overflow:hidden; border-bottom:1px solid #c4c4c4;}

.recent_heading h4 {
  color: #05728f;
  font-size: 21px;
  margin: auto;
}
.srch_bar input{ border:1px solid #cdcdcd; border-width:0 0 1px 0; width:80%; padding:2px 0 4px 6px; background:none;}
.srch_bar .input-group-addon button {
  background: rgba(0, 0, 0, 0) none repeat scroll 0 0;
  border: medium none;
  padding: 0;
  color: #707070;
  font-size: 18px;
}
.srch_bar .input-group-addon { margin: 0 0 0 -27px;}

.chat_ib h5{ font-size:15px; color:#464646; margin:0 0 8px 0;}
.chat_ib h5 span{ font-size:13px; float:right;}
.chat_ib p{ font-size:14px; color:#989898; margin:auto}
.chat_img {
  float: left;
  width: 11%;
}
.chat_ib {
  float: left;
  padding: 0 0 0 15px;
  width: 88%;
}

.chat_people{ overflow:hidden; clear:both;}
.chat_list {
  border-bottom: 1px solid #c4c4c4;
  margin: 0;
  padding: 18px 16px 10px;
}
.inbox_chat { height: 550px; overflow-y: scroll;}

.active_chat{ background:#ebebeb;}

.incoming_msg_img {
  display: inline-block;
  width: 6%;
}
.received_msg {
  display: inline-block;
  padding: 0 0 0 10px;
  vertical-align: top;
  width: 92%;
 }
 .received_withd_msg p {
  background: #ebebeb none repeat scroll 0 0;
  border-radius: 3px;
  color: #646464;
  font-size: 14px;
  margin: 0;
  padding: 5px 10px 5px 12px;
  width: 100%;
}
.time_date {
  color: #747474;
  display: block;
  font-size: 12px;
  margin: 8px 0 0;
}
.received_withd_msg { width: 100%;}
.mesgs {
  float: left;
  padding: 30px 15px 0 25px;
  width: 60%;
}

 .sent_msg p {
  background: rgb(0 123 255 / 25%) none repeat scroll 0 0;
  border-radius: 3px;
  font-size: 14px;
  margin: 0;
  padding: 5px 10px 5px 12px;
  width:96%;
}
.outgoing_msg{ overflow:hidden; margin:15px 0 15px;}
.sent_msg {
  float: right;
  align: right;
  width: 46%;
}
.input_msg_write input {
  background: rgba(0, 0, 0, 0) none repeat scroll 0 0;
  border: medium none;
  color: #4c4c4c;
  font-size: 15px;
  min-height: 48px;
  width: 100%;
}

.type_msg {border-top: 1px solid #c4c4c4;position: relative;}
.msg_send_btn {
  background: #05728f none repeat scroll 0 0;
  border: medium none;
  border-radius: 50%;
  color: #fff;
  cursor: pointer;
  font-size: 17px;
  height: 33px;
  position: absolute;
  right: 0;
  top: 11px;
  width: 33px;
}
.messaging { padding: 0 0 50px 0;}
.msg_history {
  height: 516px;
  overflow-y: auto;
}

</style>


Overwriting templates/industrialsafety_chat.html


In [39]:
from flask_ngrok import run_with_ngrok 

app = Flask(__name__) 
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER
run_with_ngrok(app)

In [40]:
@app.route("/get")
#function for the bot response
def get_bot_response():
    input_message = request.args.get('msg')
    return str(chat_response(input_message))

@app.route('/', methods=['GET','POST'])
def load_page():
  return render_template("industrialsafety_chat.html")

In [ ]:
if __name__ == "__main__":
  app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://e8cf3c367dcb.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [26/Apr/2021 03:35:34] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [26/Apr/2021 03:35:34] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [26/Apr/2021 03:35:37] "GET /get?msg=hi HTTP/1.1" 200 -
127.0.0.1 - - [26/Apr/2021 03:35:48] "GET /get?msg=what%20is%20safety%3F HTTP/1.1" 200 -
127.0.0.1 - - [26/Apr/2021 03:36:32] "GET /get?msg=which%20year%20accident%20occurred%20most%3F HTTP/1.1" 200 -
127.0.0.1 - - [26/Apr/2021 03:37:02] "GET /get?msg=which%20all%20industry%20covered%3F HTTP/1.1" 200 -
127.0.0.1 - - [26/Apr/2021 03:37:30] "GET /get?msg=what%20all%20risks%20covered%3F HTTP/1.1" 200 -
